<a href="https://colab.research.google.com/github/ChaitaliV/IITDelhi-RA/blob/main/basic_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers
!pip install sentencepiece
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.6 MB/s eta 0:00:00


In [4]:
import torch
import pandas as pd
import requests
import json
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast
import evaluate
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [ ]:
Tokenizer = T5TokenizerFast.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base", return_dict=True)
pattern_len = 256
response_len = 32
batch_size = 8
device = "cuda:0"

## Get Data

In [5]:
# Define the API URL
api_url = "https://datasets-server.huggingface.co/rows?dataset=alexandreteles%2Fmental-health-conversational-data&config=default&split=train&offset=0&length=100"

# Send an HTTP GET request to the API URL
response = requests.get(api_url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the response JSON
    data = json.loads(response.text)
    # The 'data' variable now contains the dataset content

    # You can access and work with the dataset here

else:
    print("Failed to download the dataset. Status code:", response.status_code)


In [6]:
data

{'features': [{'feature_idx': 0,
   'name': 'Context',
   'type': {'dtype': 'string', '_type': 'Value'}},
  {'feature_idx': 1,
   'name': 'Knowledge',
   'type': {'dtype': 'string', '_type': 'Value'}},
  {'feature_idx': 2,
   'name': 'Response',
   'type': {'dtype': 'string', '_type': 'Value'}}],
 'rows': [{'row_idx': 0,
   'row': {'Context': 'Hi',
    'Knowledge': 'greeting',
    'Response': 'Hello there. Tell me how are you feeling today?'},
   'truncated_cells': []},
  {'row_idx': 1,
   'row': {'Context': 'Hi',
    'Knowledge': 'greeting',
    'Response': 'Hi there. What brings you here today?'},
   'truncated_cells': []},
  {'row_idx': 2,
   'row': {'Context': 'Hi',
    'Knowledge': 'greeting',
    'Response': 'Hi there. How are you feeling today?'},
   'truncated_cells': []},
  {'row_idx': 3,
   'row': {'Context': 'Hi',
    'Knowledge': 'greeting',
    'Response': 'Great to see you. How do you feel currently?'},
   'truncated_cells': []},
  {'row_idx': 4,
   'row': {'Context': 'Hi

In [6]:
### preparing data : tokenize the text inputs and outputs, get input_ids and attention_masks

# sentences are inputs and responses are outputs
sentences = data.pattern.values
responses = data.response.values

# Adding <startofstring> and <endofstring> tokens at the beginning and end of each sentence for the model
sentences = ["<startofstring> "+ str(sentence) +" <endofstring>" for sentence in sentences]

In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
tokenizer = T5Tokenizer.from_pretrained("t5-small")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
encoding = tokenizer(
    [str(sequence) for sequence in sentences],
    padding="longest",
    max_length= 512,
    truncation=True,
    return_tensors="pt",
)

In [10]:
input_ids, attention_mask = encoding.input_ids, encoding.attention_mask

In [11]:
target_encoding = tokenizer(
    [str(response) for response in responses],
    padding="longest",
    max_length= 512,
    truncation=True,
    return_tensors="pt",
)

In [12]:
labels = target_encoding.input_ids

# replace padding token id's of the labels by -100 so it's ignored by the loss
labels[labels == tokenizer.pad_token_id] = -100

In [13]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [14]:
train_data = TensorDataset(input_ids, attention_mask, labels)

In [15]:
train_sampler = RandomSampler(train_data)

# Create a DataLoader that handles your input and output sequences and their respective masks
batch_size = 8  # Define your batch size
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [16]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [17]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']
# Separate the `weight` parameters from the `bias` parameters.
# - For the `weight` parameters, this specifies a 'weight_decay_rate' of 0.01.
# - For the `bias` parameters, the 'weight_decay_rate' is 0.0.
optimizer_grouped_parameters = [
    # Filter for all parameters which *don't* include 'bias', 'gamma', 'beta'.
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.1},

    # Filter for parameters which *do* include those.
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

In [18]:
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange

In [19]:
# The Hyperparameters for the Training Loop

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

optimizer = AdamW(optimizer_grouped_parameters,
                  lr = 3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )
# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives
# us the number of batches.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0.1, # Default value in run_glue.py
                                            num_training_steps = total_steps)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [21]:
# The Training Loop
t = []

# Store our loss and accuracy for plotting
train_loss_set = []

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):


  # Training

  # Set our model to training mode (as opposed to evaluation mode)
  model.train()

  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0

  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs['loss']
    train_loss_set.append(loss.item())
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()

    # Update the learning rate.
    scheduler.step()


    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))


Epoch:  33%|███▎      | 1/3 [00:30<01:00, 30.30s/it]

Train loss: 4.046650191387498


Epoch:  67%|██████▋   | 2/3 [00:56<00:27, 27.93s/it]

Train loss: 3.6202115725321944


Epoch: 100%|██████████| 3/3 [01:23<00:00, 27.75s/it]

Train loss: 3.4886769788811005


In [24]:
model.to('cpu')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [32]:
model.to('cpu')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [33]:
input_ids = tokenizer("How do I know if I'm unwell?", return_tensors="pt").input_ids
input_ids.to('cpu')
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Wie kann ich denken, ob ich nicht so ettiere


In [ ]:
def infer(pattern):
  pattern = "<startofstring> "+ str(pattern) +" <endofstring>"
  encoding = tokenizer(
    pattern,
    padding="longest",
    max_length=512,
    truncation=True,
    return_tensors="pt",
)
  input_ids = encoding.input_ids
  input_ids.to(device)
  input_attention = encoding.attention_mask

